In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
from utils.analysis_tools_lazimpa import *
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [ ]:
datasets = ('(3,4)', '(3,8)', '(4,4)')
n_attributes = (3, 3, 4)
n_values = (4, 8, 4)
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [3]:
datasets = ['(4,4)',]
n_values = [4,]
n_attributes = [4,]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [4]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [5]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = True
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting

### Determine vocab size

In [6]:
from collections import Counter

In [7]:
# go through all datasets
for i, d in enumerate(datasets):
    
    for run in range(5):
        # select first run
        interaction = load_interaction(paths[i],setting,run,n_epochs)
        
        messages = retrieve_messages(interaction)
        all_symbols = [symbol for message in messages for symbol in message]
        symbol_counts = Counter(all_symbols)
        print("Run ",run)
        print(symbol_counts)
        print(len(symbol_counts))

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 3876, 14: 366, 13: 363, 7: 359, 10: 303, 6: 275, 2: 260, 1: 251, 11: 246, 8: 217, 9: 207, 12: 141, 3: 134, 5: 127, 4: 103})
15


### See which messages are produced for which concepts

In [10]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [11]:
# go through all datasets
run = 0
for i, d in enumerate(datasets):
    # select run
    interaction = load_interaction(paths[i],setting,run,n_epochs)
    messages = retrieve_messages(interaction,remove_after_eos=True)
    concepts, context_conds = retrieve_concepts_context(interaction,n_values[i])

    # get random qualitative samples
    #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
    #n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
    n_fixed = 3
    #fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
    fixed_indices = [0, 1, 2]
    #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
    #fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
    fixed_values = [0, 0, 0]
    print(n_fixed, fixed_indices, fixed_values)
    #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
    # TODO: adapt this loop such that multiple indices can be fixed
    all_for_this_concept = []
    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
        if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
            for t_object in t_objects:
                if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                    all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                    fixed = t_fixed
    if len(all_for_this_concept) > 0:
        #sample = random.sample(all_for_this_concept, 20)
        sample = all_for_this_concept
        column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
        df = pd.DataFrame(sample, columns=column_names)
        path_to_save = 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv'
        print(path_to_save)
        df.to_csv(path_to_save, index=False)
        print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
    else:
        raise ValueError("sample for dataset " + str(d) + " could not be generated")

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1807, 20, 12])
3 [0, 1, 2] [0, 0, 0]
analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv
saved analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv


In [12]:
# Groupby und Aggregation for analysis using pandas
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = list(message_counts.values())
    return unique_messages, frequencies

def group_by_messages(df):
    grouped = df.groupby(['context condition','object','fixed indices'])['message'].apply(aggregate_messages).reset_index()
    grouped[['unique_messages', 'frequencies']] = pd.DataFrame(grouped['message'].tolist(), index=grouped.index)
    return grouped.drop('message', axis=1)

In [13]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv')
#group_by_messages(df) # IMPORTANT: These two messages each are the same because of eos!!!

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[0. 0. 0.],[1. 1. 1.],"[[2, 0, 13, 0], [2, 0, 0, 0]]","[60, 30]"
1,1,[0. 0. 0.],[1. 1. 1.],"[[2, 0, 13, 0], [2, 0, 0, 0]]","[50, 30]"
2,2,[0. 0. 0.],[1. 1. 1.],"[[2, 13, 0, 0]]",[100]


In [14]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[2. 3. 3.],[1. 1. 1.]all.csv')
group_by_messages(df)

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[2. 3. 3.],[1. 1. 1.],"[[6, 0, 0, 0]]",[80]
1,1,[2. 3. 3.],[1. 1. 1.],"[[6, 5, 0, 0], [6, 3, 0, 0], [6, 5, 3, 0], [5,...","[30, 20, 10, 10, 10]"
2,2,[2. 3. 3.],[1. 1. 1.],"[[6, 3, 0, 0]]",[90]


### Case study: concepts on different levels of abstraction

In [15]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [18]:
interaction = load_interaction(paths[0],setting,0,n_epochs)
messages = retrieve_messages(interaction)
concepts, context_conds = retrieve_concepts_context(interaction,n_values[0])

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1807, 20, 12])


In [19]:
concept = np.array([0., 2., 0.,]) # Warum hat das Konzept 0 in der mitte fix
fixed = np.array([0., 1., 0.])
n_fixed = 1
fixed_indices = [1,]
fixed_values = [0, 1, 2]
print(n_fixed, fixed_indices, fixed_values)
#index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
# TODO: adapt this loop such that multiple indices can be fixed
all_for_this_concept = []
for idx, (t_objects, t_fixed) in enumerate(concepts):
    #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
    if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
        for t_object in t_objects:
            if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
if len(all_for_this_concept) > 0:
    sample = random.sample(all_for_this_concept, 20)
    column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
    df = pd.DataFrame(sample, columns=column_names)
    df.to_csv('analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv', index=False)
    print('saved ' + 'analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv')
else:
    raise ValueError("sample for dataset " + str(datasets[0]) + " could not be generated")

1 [1] [0, 1, 2]
saved analysis/quali_(3,4)_lazy_context_aware_[0. 2. 0.],[0. 1. 0.].csv


In [20]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 1. 2.],[1. 1. 1.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,1045,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
1,488,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"
2,1061,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
3,1261,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
4,335,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"
5,593,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
6,1316,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
7,1061,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
8,151,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
9,1046,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"


In [21]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 1. 0.],[0. 1. 0.].csv')
df # warum ist context condition hier immer 0??? Zeigt aber dass die Objekte alle mit der selben Nachricht benannt werden. Wie ist das für die anderen Attribute und andere Values. 

,game_nr,object,fixed indices,context condition,message
0,1794,[1. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
1,1514,[2. 0. 0.],[0. 1. 0.],0,"[14, 14, 0, 0]"
2,1276,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
3,1514,[0. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
4,212,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
5,212,[1. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
6,1276,[3. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"
7,1021,[1. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"
8,1794,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
9,1276,[0. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"


In [22]:
# same attribute value, different position -> different message
#df = pd.read_csv('analysis/quali_(3,4)_standard_1,1.csv')
#df

In [23]:
#df = pd.read_csv('analysis/quali_(3,4)_standard_2,1.csv')
#df

### Dataset (3,4)

NOTE: Output contains redundant information: In the same game, the same message is used to refer to different objects because only one message is transmitted.

In [24]:
#df = pd.read_csv('analysis/quali_(3,4)_2,2.csv')
#df